<center>
<img src='./img/nsidc_logo.png'/>

# **IceFlow**
### Point Cloud Data Access
</center>

---

## Visualizing Large Data Sets
IceFlow and ICESat-2 data sets are big data sets that require some special considerations when working with them. The main constraint is memory. The average granule size is in the 10s of Megabyte for ICESat-2 and could be Gigabytes in *IceFlow* depending on the order/subsetting. 

This notebook will use some basic plotting techniques using *vaex*, *matplotlib* and *geopandas* to work effectively with point cloud data from *IceFlow* and ICESat-2 data.


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import glob
import geopandas
import pandas as pd
import h5py
import vaex
import numpy as np
from iceflow.processing import IceFlowProcessing as ifp

# filepath = 'data/atm1b_data_2020-07-10T15-32.hdf5'
# df_k = ifp.get_common_dictionary('ATM')

filepath = 'data/twaties-test-GLAH06-2000-2010.h5'
df_key = ifp.get_common_dictionary('GLAS')

## Loading Data with H5PY

In [ ]:
%%time

f = h5py.File(filepath, 'r')
print(list(f.keys()))

## Vaex Decimating Big Datafames 

[*Vaex*](https://vaex.io/docs/index.html) is a python library for "lazy" processing (computing on the fly, no wasted ram), to visualize and explore big tabular data sets. Using *IceFlow*'s common data dictionary, we are going to create a "decimated" dataframe with only 1/100 of the size of the original dataframe to help us plot the data faster. 


In [ ]:
%%time

df = vaex.open(filepath)
# We're parsing the utc_datetime from IceFlow into a data type that vaex understands.
df['date'] = df.utc_datetime.values.astype('datetime64[ns]')
# my_df = df['longitude', 'latitude', 'elevation', 'date']
# Note that we need a common dictionary because in GLAH06 elevation is d_elev and in ICESat-2 is called elevation! 
my_df = df[df_key['latitude'], df_key['longitude'], df_key['elevation'], 'date']
# vaex.vrange() is like numpy.arange but uses 0-memory no matter the length.
# This is to down-sample the data for dataviz see: https://github.com/vaexio/vaex/issues/911
df.add_column('index', vaex.vrange(0, len(df)))
# We are going to create a "decimated" dataframe with only 1/100 of the size of the original to plot the big picture faster.
df_decimated = df[(df.index % 100 == 0)]
my_df.describe()
display(my_df)

## Visualizing the Big Picture

Here we will quickly plot the decimated dataframe to view coverage and cross-over locations.

In [ ]:
my_df.widget.heatmap(my_df[df_key['longitude']], 
               my_df[df_key['latitude']],
               what=vaex.stat.mean(my_df[df_key['elevation']]),
               shape=512, 
               figsize=(10,6),
               limits='minmax',
               colormap='inferno')

Next we can use *matplotlib* and *cartopy* to plot the decimated data in an interactive map widget.

In [ ]:
%matplotlib widget
import vaex
from ipywidgets import widgets
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

plt.figure(figsize=(10,8), dpi= 90)
ax = plt.axes(projection=ccrs.SouthPolarStereo(central_longitude=0)) 
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.set_extent([-180, 180, -65, -90], ccrs.PlateCarree())
plt.scatter(df_decimated[df_key['longitude']].values,
            df_decimated[df_key['latitude']].values,
            c=df_decimated[df_key['elevation']].values,
            cmap='viridis',
            vmin=100,vmax=200,
            transform=ccrs.PlateCarree())
plt.colorbar(label='elevation', shrink=0.5, extend='both')

## "Flying" with the Sensor

This plotting example allows us to "fly" along the decimated point cloud data track in a 3D graph environment.

In [ ]:
%matplotlib widget
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt


fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(70, 70)

#If the data granule(s) is big enough (1+GB), use the decimated dataframe.
# df = df_decimated

def plot_func(alontrack):
    step = 5000 # same as density
    m = int(alontrack * step)
    ax.clear()
    ax.scatter(df[df_key['longitude']].values[m:m+step],
               df[df_key['latitude']].values[m:m+step],
               df[df_key['elevation']].values[m:m+step],
               c=df[df_key['elevation']].values[m:m+step],
               cmap='viridis', s=1)
    ax.axis('tight')


interact(plot_func, alontrack = widgets.FloatSlider(value=0,
                                                    description='Along Track Steps',
                                                    min=0,
                                                    max=90,
                                                    step=0.3,
                                                    layout={'width': '100%'}))